Marcin Wardyński  
wtorek, 9:45

## Laboratorium 7
### 7.2 DBN

In [67]:
import importlib
import lab7_utils as utils
importlib.reload(utils)

seed = 42

In [68]:
from sklearn.base import TransformerMixin, BaseEstimator

class FrozenRBM(TransformerMixin, BaseEstimator):
    def __init__(self, rbm):
        self.rbm = rbm

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        return self.rbm.transform(X)

In [69]:
from sklearn.base import clone
from sklearn.pipeline import Pipeline

def wrap_rbm_snapshots(rbms_snaps):
    rbm_tuples = []
    for i in range(len(rbms_snaps)):
        rbm_tuples.append((f"rbm_L{i+1}", rbms_snaps[i]))

    return rbm_tuples


def create_dbn(hidden_dims, rbm_base, log_reg_base, X_train, y_train):
    rbm_snapshots = []
    log_regs = []

    log_reg = clone(log_reg_base)
    pipeline = Pipeline([("log_reg", log_reg)])
    pipeline.fit(X_train, y_train)
    log_regs.append(log_reg)


    for hidden_dim in hidden_dims:
        log_reg = clone(log_reg_base)
        rbm = clone(rbm_base)
        rbm.n_components = hidden_dim


        pipeline_def = []
        pipeline_def.extend(wrap_rbm_snapshots(rbm_snapshots))
        pipeline_def.append((f"rbm_L{len(rbm_snapshots)+1}", rbm))
        pipeline_def.append(("log_reg", log_reg))
       
        pipeline = Pipeline(pipeline_def)
        pipeline.fit(X_train, y_train)

        rbm_snapshots.append(FrozenRBM(rbm))
        log_regs.append(log_reg)
    
    return rbm_snapshots, log_regs


In [75]:
from sklearn.metrics import accuracy_score, classification_report

def test_dbn(rbms, log_regs, X_test, y_test, print_report):
    accuracies = []
    for i in range(len(rbms)+1):
        pipeline_def = []
        pipeline_def.extend(wrap_rbm_snapshots(rbms[:i]))
        pipeline_def.append(("log_reg", log_regs[i]))

        pipeline = Pipeline(pipeline_def)
        y_pred = pipeline.predict(X_test)

        accuracy = accuracy_score(y_test, y_pred)
        accuracies.append(accuracy)

        if print_report:
            print(f"Warstwy modelu: {pipeline.named_steps.keys()}")
            print(classification_report(y_test, y_pred))
            
    return accuracies

In [71]:
from sklearn.linear_model import LogisticRegression
from sklearn.neural_network import BernoulliRBM

def check_dbn(dataset_name, model_params, l1_h_dim, l2_h_dim, l3_h_dim, print_report):
    X_train, X_test, y_train, y_test = utils.get_dataset(dataset_name)

    rbm_base = BernoulliRBM(learning_rate=model_params['learning_rate'], batch_size=model_params['batch_size'], random_state=seed)
    log_reg_base = LogisticRegression(max_iter=model_params['max_iter'], solver=model_params['solver'], C=model_params['C'])

    hidden_dims = [l1_h_dim, l2_h_dim, l3_h_dim]

    rbms, log_regs = create_dbn(hidden_dims, rbm_base, log_reg_base, X_train, y_train)

    accuracies = test_dbn(rbms, log_regs, X_test, y_test, print_report)

    return max(accuracies)

In [72]:
import optuna

def objective(trial, dataset_name, model_params):

    l1_h_dim = trial.suggest_categorical('l1_h_dim', [128, 256, 512])
    l2_h_dim = trial.suggest_categorical('l2_h_dim', [64, 128, 192])
    l3_h_dim = trial.suggest_categorical('l3_h_dim', [32, 64, 96])
    
    return check_dbn(dataset_name, model_params, l1_h_dim, l2_h_dim, l3_h_dim, print_report=False)

def optimize_dbn(dataset_name, model_params, n_trials):

    study = optuna.create_study(direction='maximize')
    study.optimize(lambda trial: objective(trial, dataset_name, model_params), n_trials=n_trials, show_progress_bar=True)

    print(f"Best parameters: {study.best_params}")
    print_best_dbn_summary(dataset_name, model_params, study.best_params)

def print_best_dbn_summary(dataset_name, model_params, l_h_dims):
    check_dbn(dataset_name, model_params, l_h_dims['l1_h_dim'], l_h_dims['l2_h_dim'], l_h_dims['l3_h_dim'], print_report=True)

In [73]:
n_trials = 1
solver = 'lbfgs'

In [74]:
model_params = {
    'C': 0.5,
    'solver': solver,
    'max_iter': 1000,
    'batch_size': 20,
    'learning_rate': 0.05,
}

optimize_dbn(utils.Dataset_Select.MNIST.value, model_params, n_trials)

[I 2025-01-04 18:40:08,849] A new study created in memory with name: no-name-e2acce35-b540-449d-8bd8-2d58fae7220a
Best trial: 0. Best value: 0.9684: 100%|██████████| 1/1 [02:26<00:00, 146.31s/it]


[I 2025-01-04 18:42:35,154] Trial 0 finished with value: 0.9684 and parameters: {'l1_h_dim': 512, 'l2_h_dim': 64, 'l3_h_dim': 96}. Best is trial 0 with value: 0.9684.
Best parameters: {'l1_h_dim': 512, 'l2_h_dim': 64, 'l3_h_dim': 96}
dict_keys(['log_reg'])
              precision    recall  f1-score   support

           0       0.94      0.97      0.95       980
           1       0.96      0.98      0.97      1135
           2       0.92      0.89      0.90      1032
           3       0.90      0.91      0.90      1010
           4       0.92      0.92      0.92       982
           5       0.88      0.85      0.87       892
           6       0.94      0.94      0.94       958
           7       0.92      0.92      0.92      1028
           8       0.87      0.87      0.87       974
           9       0.90      0.90      0.90      1009

    accuracy                           0.92     10000
   macro avg       0.92      0.92      0.92     10000
weighted avg       0.92      0.92      

In [ ]:
model_params = {
    'C': 0.5,
    'solver': solver,
    'max_iter': 5000,
    'batch_size': 10,
    'learning_rate': 0.01,
}

optimize_dbn(utils.Dataset_Select.F_MNIST.value, model_params, n_trials)

In [ ]:
model_params = {
    'C': 1.0,
    'solver': solver,
    'max_iter': 1000,
    'batch_size': 10,
    'learning_rate': 0.1,
}

optimize_dbn(utils.Dataset_Select.K_MNIST.value, model_params, n_trials)

In [ ]:
model_params = {
    'C': 0.5,
    'solver': solver,
    'max_iter': 1000,
    'batch_size': 20,
    'learning_rate': 0.1,
}

optimize_dbn(utils.Dataset_Select.KUZ_49.value, model_params, n_trials)